<a href="https://colab.research.google.com/github/rakesh4482/transformers/blob/main/QuestionAnswer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#question = "What is the capital of USA? Tell more about capital of USA."
#question ="What is Python programming language? Explain in detail."
question ="Whos is the father of AI? Explain in detail."
#question =input(">"))
inputs = tokenizer("Question: " + question, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=500)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Aristotle was the father of mathematics . Aristotle was the first to prove that the universe was made of matter and that the laws of physics were the laws of the universe. Aristotle was the first to prove that the universe was made of matter and that the laws of physics were the laws of the universe. So the answer is Aristotle.


**Training of the data now** ✔

In [ ]:
!pip install transformers datasets sentencepiece accelerate

In [ ]:
qa_dataset = [
    {"question": "What is Python used for?", "answer": "Python is used for web development, data analysis, machine learning, and automation."},
    {"question": "How does Django handle routing?", "answer": "Django uses URL patterns defined in urls.py to map URLs to views."},
    {"question": "What is a Transformer model?", "answer": "A Transformer is a neural network architecture designed for handling sequential data, especially in NLP."},
    {"question": "How to deploy a Django app?", "answer": "You can deploy a Django app using platforms like Heroku, Render, or AWS with proper static file settings."},
    {"question": "What is Hugging Face used for?", "answer": "Hugging Face provides tools and models for natural language processing tasks like text classification and question answering."},
    {"question": "What is BERT?", "answer": "BERT is a transformer-based model that understands context in text by processing words bidirectionally."},
    {"question": "How to fine-tune a BERT model?", "answer": "You fine-tune BERT by training it on a labeled dataset using a task-specific head like classification or QA."},
    {"question": "What is tokenization in NLP?", "answer": "Tokenization is the process of breaking text into smaller units like words or subwords for processing."},
    {"question": "How to track Udemy income using Python?", "answer": "You can use Python to scrape or import CSV reports, then analyze earnings with pandas and visualize with matplotlib."},
    {"question": "What is the role of attention in Transformers?", "answer": "Attention allows the model to focus on relevant parts of the input sequence when generating output."},
    {"question": "How to use Google Colab for training?", "answer": "Google Colab provides free GPU access and lets you run Jupyter notebooks in the cloud for training models."},
    {"question": "What is the difference between extractive and generative QA?", "answer": "Extractive QA pulls answers from a context, while generative QA writes answers from learned knowledge."},
    {"question": "How to create a modular game in Django?", "answer": "Use Django apps to separate game logic, templates for UI, and views to handle game state transitions."},
    {"question": "What is a custom BERT model?", "answer": "A custom BERT model is a fine-tuned version of BERT trained on domain-specific data for better performance."},
    {"question": "How to automate tax reconciliation?", "answer": "Use Python to parse income reports, match them with expenses, and generate summaries for tax filing."},
    {"question": "What is the Hugging Face Hub?", "answer": "The Hugging Face Hub is a platform to share, discover, and deploy machine learning models and datasets."},
    {"question": "How to write devotional lyrics?", "answer": "Start with a theme, use poetic structure, and express emotion and reverence through metaphors and rhythm."},
    {"question": "What is Git used for?", "answer": "Git is a version control system used to track changes in code and collaborate with others."},
    {"question": "How to build a privacy-conscious web app?", "answer": "Use minimal tracking, secure authentication, encrypted data storage, and transparent privacy policies."},
    {"question": "What is Azure ML?", "answer": "Azure ML is a cloud-based platform for building, training, and deploying machine learning models."}
]


In [ ]:
from datasets import Dataset

# Convert list of dicts to Hugging Face Dataset
dataset = Dataset.from_list(qa_dataset)

# Split into train/test
dataset = dataset.train_test_split(test_size=0.2)


# *********  Load FLAN-T5 and Tokenizer ************

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#  ***************  prepare data set  *********************

def preprocess(example):
    input_text = "Question: " + example["question"]
    target_text = example["answer"]
    model_inputs = tokenizer(input_text, max_length=128, truncation=True)
    labels = tokenizer(target_text, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess)



Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

We will do Training of the DataSet ✅

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-qa",

    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    logging_dir="./logs",
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Test the Model now

In [ ]:
question = "What is Git ?"
inputs = tokenizer("Question: " + question, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


a website that provides a variety of free software, including a website that provides information and tutorials for beginners, and a website that provides a variety of tutorials and tutorials for children, including a website that provides 
